## Импортирование библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from numpy import mean
from sklearn.linear_model import LinearRegression

In [2]:
boston = load_boston()
X = boston["data"]
y = boston["target"]
np.random.seed(42)

### построение бутстрап-выборки из выборки x,y и индексов idx

In [4]:
def build_bootstrap_samples(x, y, idx):
    chosen_indexes = np.random.choice(idx, size=idx.shape[0], replace=True)
    chosen_x = x[chosen_indexes]
    chosen_y = y[chosen_indexes]
    return chosen_x, chosen_y, chosen_indexes

### вычисление ошибки, смещения и разброса модели путем построения бутсрап-выборок

In [7]:
def evaluate_variance_bias_tradeoff(model, model_label, x, y, n_rounds):
    predictions = np.zeros((n_rounds, y.shape[0]))
    indexes = np.arange(y.shape[0])
    
    for i in range(n_rounds):
        x_bootstrap, y_bootstrap, indexes_bootstrap = build_bootstrap_samples(x, y, indexes)
        indexes_out_of_bag = np.setdiff1d(indexes, indexes_bootstrap)
        x_out_of_bag = x[indexes_out_of_bag]
        predictions[i][indexes_out_of_bag] = model.fit(x_bootstrap, y_bootstrap).predict(x_out_of_bag)
        predictions[i][indexes_bootstrap] = np.NaN
        
    error = np.nanmean((predictions - y)**2)
    variance = np.mean(np.nanvar(predictions, axis=0, ddof=1))
    bias = np.mean((np.nanmean(predictions, axis=0) - y)**2)
    return predictions, pd.DataFrame( dict(error=[error], variance=[variance], bias=[bias], model=[model_label]) )

In [8]:
BOOTSTRAP_ROUNDS = 100

### вычисление предсказаний и ошибки, разброса, смещения для линейной регрессии

In [12]:
lr_predictions, lr_scores = evaluate_variance_bias_tradeoff(model=LinearRegression(), model_label='LinearRegression', x=X, y=y, n_rounds=BOOTSTRAP_ROUNDS)

### вычисление предсказаний и ошибки, разброса, смещения для решающего дерева

In [9]:
dt_predictions, dt_scores = evaluate_variance_bias_tradeoff(model=DecisionTreeRegressor(), model_label='DecisionTree', x=X, y=y, n_rounds=BOOTSTRAP_ROUNDS)

### вычисление предсказаний и ошибки, разброса, смещения для случайного леса

In [10]:
rf_predictions, rf_scores = evaluate_variance_bias_tradeoff(model=RandomForestRegressor(), model_label='RandomForest', x=X, y=y, n_rounds=BOOTSTRAP_ROUNDS)

In [14]:
results = pd.concat([lr_scores, dt_scores, rf_scores]).set_index('model')
results

,error,variance,bias
model,,,
LinearRegression,24.245616,0.891447,23.707356
DecisionTree,24.045279,13.714874,10.607370
RandomForest,12.642663,2.459253,10.385976


### сумма разброса и смещения очень близка к общей ошибке, отличия обусловлены случайным шумом

### Исходя из полученных результатов
- Linear Regression: смещение значительно превышает разброс, возможно недообучение модели;
- Decision Tree: разброс и смещение близки друг к другу, присутствует правильный баланс между недообучением и переобучением;
- Random Forest: смещение намного больше разброса, алгоритм недообучен, при этом общая ошибка значительно меньше, чем у других моделей линейной регрессии и решающего дерева.